In [1]:
from openai import AzureOpenAI
from dotenv import load_dotenv
import pandas as pd
import os
from jinja2 import Environment, FileSystemLoader
from pathlib import Path
from tqdm.notebook import tqdm
import time
import sys
import importlib

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

from utils.utils import convert_types, eval
from utils.vars import DATA_DIR, EXCEPT_FILES, QUESTION_PATH

load_dotenv()

True

In [2]:
# read questions
df_questions = pd.read_csv(QUESTION_PATH)

# get the client object
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-10-01-preview",  # different from assistant
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
)

## Using the same prompt in the function_instruction.jinja2 file

- OpenAI models cannot run the code but can generate the code correctly
- The #tokens are a lot 
- The execution time is much longer: 20s for one question for melborune_temp
- Cannot run ny_taxi.csv due to token limit
- data is not corrected in the code
- The code overall looks right but does not round to 2 as stated in the prompt

- [TODO] Is there any other way to pass the dataframe

In [3]:
# read the prompt
instruction = (
    Environment(loader=FileSystemLoader("."))
    .get_template("prompts/prompt.jinja2")
    .render()
)

df_result = []

for file_path in Path(DATA_DIR).glob("*.csv"):
    if file_path.name in EXCEPT_FILES + ["nyc_taxi.csv"]:
        continue
    print(f"file: {file_path.name}")

    # read the data
    df = pd.read_csv(file_path)

    # call openai
    for _, row in tqdm(df_questions.iterrows(), total=len(df_questions)):
        question = row["question"]
        answer_true = row[Path(file_path).name]

        start_time = time.time()

        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": instruction},
                {
                    "role": "user",
                    "content": f"Here is the dataset in the markdown format. {df.to_markdown()}",
                },
                {"role": "user", "content": question},
            ],
            temperature=0,
            top_p=1,
            seed=42,
        )

        code = response.choices[0].message.content
        with open("tmp.py", "w") as f:
            f.write(code.replace("```python\n", "").replace("```", ""))

        # execute the code

        try:
            import tmp

            importlib.reload(tmp)
            answer_pred = convert_types(tmp.main()["output"])
        except:
            answer_pred = None

        df_result.append(
            {
                **response.usage.to_dict(),
                "question": question,
                "execution_time_s": round(time.time() - start_time, 2),
                "file": file_path.name,
                "answer_pred": answer_pred,
                "answer_true": convert_types(answer_true),
                "messages": [],
                "steps": [],
            }
        )


df_result = pd.DataFrame(df_result)

file: air_passengers.csv


  0%|          | 0/16 [00:00<?, ?it/s]

{'output': 104}
{'output': 104}
{'output': 622}
{'output': 280.3}
{'output': 265.5}
{'output': 119.97}
{'output': 180.0}
{'output': 360.5}
{'output': 0}
{'output': 0}
{'output': 0}
{'output': 144}
{'output': '1949-01-01'}
{'output': '1960-12-01'}
{'output': 'MS'}
file: melbourne_temp.csv


  0%|          | 0/16 [00:00<?, ?it/s]

In [6]:
# eval
df_result = pd.DataFrame(df_result)

# loop through each file
eval(df=df_result, details=True)

File: air_passengers.csv; Accuracy: 1.0
File: melbourne_temp.csv; Accuracy: 0.1875
question: What is the min of the target variable?
answer_pred: None
answer_true: 0.0
messages: 
steps: []
**************************************************
question: What is the max of the target variable?
answer_pred: None
answer_true: 26.3
messages: 
steps: []
**************************************************
question: What is the mean of the target variable?
answer_pred: None
answer_true: 11.18
messages: 
steps: []
**************************************************
question: What is the medium of the target variable?
answer_pred: None
answer_true: 11.0
messages: 
steps: []
**************************************************
question: What is the standard deviation of the target variable?
answer_pred: None
answer_true: 4.07
messages: 
steps: []
**************************************************
question: What is the Q1 of the target variable?
answer_pred: None
answer_true: 8.3
messages: 
steps: []
***

In [5]:
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(df_result.groupby(["file"]).describe())

completion_tokens                                        \
                               count       mean          std   min     25%   
file                                                                         
air_passengers.csv              16.0  1409.9375   559.016931  15.0  1384.5   
melbourne_temp.csv              16.0  3331.5625  1643.492378  15.0  4096.0   

                                           prompt_tokens                      \
                       50%     75%     max         count      mean       std   
file                                                                           
air_passengers.csv  1650.0  1666.5  1786.0          16.0   2497.25  1.732051   
melbourne_temp.csv  4096.0  4096.0  4096.0          16.0  69134.25  1.732051   

                                                                 total_tokens  \
                        min       25%      50%      75%      max        count   
file                                                                            
air_passengers.csv   2494.0   2496.75   2497.0   2498.0   2500.0         16.0   
melbourne_temp.csv  69131.0  69133.75  69134.0  69135.0  69137.0         16.0   

                                                                         \
                          mean          std      min       25%      50%   
file                                                                      
air_passengers.csv   3907.1875   560.374544   2509.0   3881.50   4147.5   
melbourne_temp.csv  72465.8125  1644.774968  69146.0  73229.75  73230.0   

                                     execution_time_s                         \
                        75%      max            count        mean        std   
file                                                                           
air_passengers.csv   4166.0   4284.0             16.0   61.485625  31.711766   
melbourne_temp.csv  73231.0  73233.0             16.0  102.703125  76.344308   

                                                             
                     min      25%     50%       75%     max  
file                                                         
air_passengers.csv  0.83  43.9850  61.655   79.2125  123.37  
melbourne_temp.csv  1.06  83.6225  90.565  112.3750  297.47